In [ ]:
#This is the random parameter test 
#It randomly chooses 100 different combinations of the buy and sell paramaters
#Then backtests the strat with these parameters for the whole s&p 500 over a 1y period
#And returns the portfolio return and sharpe ratio

#3726s for 100 portfolios (an hour for 100)

In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pytickersymbols import PyTickerSymbols

warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

pd.options.mode.chained_assignment = None  # default='warn'

In [ ]:
# Download stock data using yfinance
def get_stock_data(ticker, period):
    data = yf.download(ticker, period = period, interval = "1d")
    return data

In [ ]:
# Calculate RSI manually using pandas
def calculate_rsi(data, rsi_period):
    delta = data['Adj Close'].diff(1)
    gain = (delta.where(delta > 0, 0)).rolling(window=rsi_period).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=rsi_period).mean()
    rs = gain / loss
    data['RSI'] = 100 - (100 / (1 + rs))
    return data

In [ ]:
# Implement the trading strategy with stop-sell condition
def trading_strategy(data, buy, sell, days, rsi_period):
    data = calculate_rsi(data, rsi_period)
    
    position = 0  # 1 means we own the stock, 0 means we don't
    holding_period = 0  # Track how long we’ve held the position
    max_holding_days = days  # Stop-sell after 5 days
    buy_price = 0
    data['Signal'] = 0

    for i in range(1, len(data)):
        if data['RSI'][i-1] < buy and data['RSI'][i] >= buy and position == 0:
            # Buy signal: RSI crosses from below 51 to above 51
            position = 1
            holding_period = 1  # Reset holding period counter on buy
            buy_price = data['Adj Close'][i]
            data['Signal'][i] = 1  # 1 for buy
            #print(f"Bought at {buy_price} on {data.index[i]}")
        
        elif position == 1:
            holding_period += 1
            
            # Sell signal: RSI crosses from below 71 to above 71 or stop-sell after 5 days
            if (data['RSI'][i-1] < sell and data['RSI'][i] >= sell) or holding_period > max_holding_days:
                position = 0
                sell_price = data['Adj Close'][i]
                data['Signal'][i] = -1  # -1 for sell
                #print(f"Sold at {sell_price} on {data.index[i]} (held for {holding_period - 1} days)")
                holding_period = 0  # Reset the holding period after selling
    
    return data

In [ ]:
# Backtest the strategy
def backtest(data):
    # Create Position column where we hold the stock only between buy and sell signals
    data['Position'] = 0  # Initialize the position as no stock held

    for i in range(1, len(data)):
        if data['Signal'][i] == 1:  # Buy signal
            data['Position'][i] = 1
        elif data['Signal'][i] == -1:  # Sell signal
            data['Position'][i] = 0
        else:
            data['Position'][i] = data['Position'][i-1]  # Continue holding or not

    # Calculate daily returns
    data['Daily Return'] = data['Adj Close'].pct_change()
    
    # Strategy returns only when we're in position
    data['Strategy Return'] = data['Daily Return'] * data['Position'].shift(1)
    
    # Set returns to 0 for days when not holding a position
    data['Strategy Return'].fillna(0, inplace=True)

    # Calculate cumulative returns
    data['Cumulative Market Return'] = (1 + data['Daily Return']).cumprod()
    data['Cumulative Strategy Return'] = (1 + data['Strategy Return']).cumprod()
    
    return data

In [ ]:
def backtest_portfolio(pulled_data, symbols, buy, sell, days, rsi_period):
    portfolio_returns = pd.DataFrame()

    for symbol in symbols:
        #print(f"Processing {symbol}...")
        try:
            individual_stock_data = pulled_data[symbol]
            if len(individual_stock_data) == 0:  # Skip symbols with no data
                continue
            
            individual_stock_data = trading_strategy(individual_stock_data, buy, sell, days, rsi_period)
            individual_stock_data = backtest(individual_stock_data)
            
            # Store strategy return for each stock
            portfolio_returns[symbol] = individual_stock_data['Strategy Return']
        except Exception as e:
            print(f"Error processing {symbol}: {e}")

    # Average the returns of all stocks to simulate equal weighting
    portfolio_returns['Portfolio Return'] = portfolio_returns.mean(axis=1)
    
    # Calculate cumulative returns for the portfolio
    portfolio_returns['Cumulative Portfolio Return'] = (1 + portfolio_returns['Portfolio Return']).cumprod()

    total_strategy_return = (portfolio_returns['Cumulative Portfolio Return'].iloc[-1] - 1)*100
    total_strategy_return = "{:.2f}%".format(total_strategy_return)

    days = 252
    risk_free_rate=0.045
    
    #Sharpe Ratio
    annualized_return = ((1 + portfolio_returns['Portfolio Return'].mean()) ** days) - 1
    annualized_volatility = portfolio_returns['Portfolio Return'].std() * np.sqrt(days)
    sharpe_ratio = (annualized_return - risk_free_rate) / annualized_volatility

    return total_strategy_return, sharpe_ratio

In [ ]:
def main_portfolio(pulled_data, symbols, buy, sell, days, rsi_period):
    # Backtest the portfolio
    total_return, sharpe_ratio = backtest_portfolio(pulled_data, symbols, buy, sell, days, rsi_period)

    new_row = pd.DataFrame({'Buy': [buy], 'Sell': [sell],'Max Holding Days': [days], 'Rsi Period': [rsi_period] , 'Returns': [total_return], 'Sharpe': [sharpe_ratio], })
               
    return new_row


In [ ]:
def generate_portfolios(num_portfolios):
    #parameters = pd.DataFrame({'Buy':[], 'Sell':[], 'Returns':[], 'Sharpe':[]})
    #parameters.set_index('Buy', inplace=True)
    parameters = pd.DataFrame(columns=['Buy', 'Sell', 'Max Holding Days', 'Rsi Period', 'Returns', 'Sharpe'])

    stock_data = PyTickerSymbols()
    sp500_symbols = stock_data.get_stocks_by_index('S&P 500')
    symbols = [stock['symbol'] for stock in sp500_symbols]  # S&P 500 stock symbols
    #symbols = ["NVDA", "AAPL", "COIN", "MSTR"]

    pulled_data = yf.download(symbols, period = "1y", interval = "1d", group_by="ticker") #Pulling all the data for all the stocks

    
    for i in range(num_portfolios):
        buy = np.random.randint(50, 101)
        sell = np.random.randint(buy + 1, 121)
        days = np.random.randint(3, 9)
        rsi_period = np.random.choice([7, 14, 30])
        new_row = main_portfolio(pulled_data, symbols, buy, sell, days, rsi_period)
        parameters = pd.concat([parameters, new_row])

    return parameters


In [ ]:
df = generate_portfolios(100)
df

In [ ]:
df.to_csv('Parameters test s&p500 v1.csv', index=True)